In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn

import torch

cuda = torch.cuda.is_available()
print("GPU:", cuda)



GPU: True


In [2]:
import os
import cv2  # OpenCV para leer imágenes

# Ruta de la carpeta del dataset
dataset_folder = 'F:\\PlantVillage-Dataset\\PlantVillage-Dataset\\raw\\grayscale'

# Lista de carpetas específicas que deseas cargar
carpetas_seleccionadas = [
    'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___Target_Spot',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___healthy',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold'
]

# Lista para almacenar las matrices de las imágenes
imagenes = []

for carpeta in carpetas_seleccionadas:
    ruta_carpeta = os.path.join(dataset_folder, carpeta)

    if os.path.exists(ruta_carpeta) and os.path.isdir(ruta_carpeta):
        archivos_en_carpeta = os.listdir(ruta_carpeta)
        print(f"Archivos en {carpeta}: {len(archivos_en_carpeta)}")

        for archivo in archivos_en_carpeta:
            ruta_archivo = os.path.join(ruta_carpeta, archivo)

            # Leer la imagen usando OpenCV
            imagen = cv2.imread(ruta_archivo)
            imagenes.append(imagen)

    else:
        print(f"La carpeta {carpeta} no existe en el dataset.")

print("Total de archivos de imagen encontrados:", len(imagenes))

# Ahora tienes las imágenes almacenadas en la lista "imagenes".
# Puedes realizar operaciones y manipulaciones adicionales según tus necesidades.

Archivos en Tomato___Septoria_leaf_spot: 1771
Archivos en Tomato___Spider_mites Two-spotted_spider_mite: 1676
Archivos en Tomato___Target_Spot: 1404
Archivos en Tomato___Tomato_mosaic_virus: 373
Archivos en Tomato___Tomato_Yellow_Leaf_Curl_Virus: 5357
Archivos en Tomato___Bacterial_spot: 2127
Archivos en Tomato___Early_blight: 1000
Archivos en Tomato___healthy: 1591
Archivos en Tomato___Late_blight: 1909
Archivos en Tomato___Leaf_Mold: 952
Total de archivos de imagen encontrados: 18160


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Suponiendo que tienes las imágenes almacenadas en la lista `imagenes`

# Define una clase personalizada para el conjunto de datos
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        # Si necesitas aplicar transformaciones a las imágenes, hazlo aquí
        if self.transform:
            image = self.transform(image)
        return image



In [6]:
# Conversión de la lista de imágenes a un formato compatible con PyTorch Dataset
custom_dataset = CustomDataset(imagenes)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
train_set, test_set = train_test_split(custom_dataset, test_size=0.2, random_state=42)

# DataLoader para manejar los datos de entrenamiento y prueba
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

# Define la arquitectura de la red neuronal convolucional
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Se ajusta al tamaño de la imagen (255x255)
        self.fc2 = nn.Linear(128, 10)  # 10 clases diferentes en el conjunto de datos

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(-1, 32 * 64 * 64)  # Se ajusta al tamaño de la imagen (255x255)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Inicializa la red neuronal
model = SimpleCNN()

# Verifica si la GPU está disponible y mueve el modelo a GPU si es así
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs = data.to(device, dtype=torch.float)
        optimizer.zero_grad()

        outputs = model(inputs)
        # Suponiendo que tienes etiquetas para tus datos
        # Asegúrate de tener las etiquetas correspondientes a tus imágenes
        # por ejemplo, 'train_labels' serían las etiquetas de tus datos de entrenamiento
        # reemplaza 'train_labels' con las etiquetas reales de tus datos
        labels = imagenes[i].to(device, dtype=torch.long)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Imprimir cada 10 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], "
                  f"Step [{i + 1}/{len(train_loader)}], "
                  f"Loss: {running_loss / 10}")
            running_loss = 0.0

print("Entrenamiento finalizado.")

# Evaluar el modelo con los datos de prueba
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs = data.to(device, dtype=torch.float)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # Suponiendo que 'test_labels' son las etiquetas de tus datos de prueba
        # Reemplaza 'test_labels' con las etiquetas reales de tus datos
        correct += (predicted == test_labels).sum().item()

accuracy = correct / total
print(f"Exactitud en los datos de prueba: {accuracy * 100:.2f}%")

RuntimeError: Given groups=1, weight of size [16, 3, 3, 3], expected input[32, 256, 256, 3] to have 3 channels, but got 256 channels instead